In [1]:
!unzip "/content/drive/My Drive/Docknet.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Docknet/train/notdocks/25385_28836_19617.jpg  
  inflating: Docknet/train/notdocks/25389_19260_16912.jpg  
  inflating: Docknet/train/notdocks/25420_22670_28628.jpg  
  inflating: Docknet/train/notdocks/25440_7070_5910.jpg  
  inflating: Docknet/train/notdocks/25511_29358_6754.jpg  
  inflating: Docknet/train/notdocks/25558_9808_22289.jpg  
  inflating: Docknet/train/notdocks/25637_30243_9292.jpg  
  inflating: Docknet/train/notdocks/25688_6516_2283.jpg  
  inflating: Docknet/train/notdocks/25719_5270_12174.jpg  
  inflating: Docknet/train/notdocks/25739_14716_28235.jpg  
  inflating: Docknet/train/notdocks/25757_5051_4253.jpg  
  inflating: Docknet/train/notdocks/25757_6463_17255.jpg  
  inflating: Docknet/train/notdocks/25768_7898_25172.jpg  
  inflating: Docknet/train/notdocks/25893_12380_4369.jpg  
  inflating: Docknet/train/notdocks/25927_25840_13436.jpg  
  inflating: Docknet/train/notdocks/25962_29101_23776.jpg  
  

In [2]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

#Loads the VGG16 model 
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 2s 0us/step


In [3]:
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D True
2 Conv2D True
3 MaxPooling2D True
4 Conv2D True
5 Conv2D True
6 MaxPooling2D True
7 Conv2D True
8 Conv2D True
9 Conv2D True
10 MaxPooling2D True
11 Conv2D True
12 Conv2D True
13 Conv2D True
14 MaxPooling2D True
15 Conv2D True
16 Conv2D True
17 Conv2D True
18 MaxPooling2D True


In [4]:
from keras.applications import VGG16

# VGG16 was designed to work on 224 x 224 pixel input images sizes
img_rows = 224
img_cols = 224 

# Re-loads the VGG16 model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in vgg16.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [0]:
def addTopModel(bottom_model, num_classes, D=256):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""
    top_model = bottom_model.output
    top_model = Flatten(name = "flatten")(top_model)
    top_model = Dense(D, activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_classes, activation = "softmax")(top_model)
    return top_model

In [6]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addTopModel(vgg16, num_classes)

model = Model(inputs=vgg16.input, outputs=FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'Docknet/train'
validation_data_dir = 'Docknet/valid'

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 6027 images belonging to 2 classes.
Found 670 images belonging to 2 classes.


In [12]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
                   
checkpoint = ModelCheckpoint("weed_detection_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_accuracy', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# Note we use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

nb_train_samples = 6027
nb_validation_samples = 670
epochs = 10
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

model.save("weed_detection_saved_model_vgg.h5")

Epoch 1/10
376/376 [==============================] - 81s 216ms/step - loss: 0.3398 - accuracy: 0.8766 - val_loss: 0.1333 - val_accuracy: 0.8512

Epoch 00001: val_loss improved from inf to 0.13334, saving model to weed_detection_vgg.h5
Epoch 2/10
376/376 [==============================] - 80s 214ms/step - loss: 0.3370 - accuracy: 0.8789 - val_loss: 0.1627 - val_accuracy: 0.8244

Epoch 00002: val_loss did not improve from 0.13334
Epoch 3/10
376/376 [==============================] - 80s 212ms/step - loss: 0.3272 - accuracy: 0.8827 - val_loss: 0.1355 - val_accuracy: 0.9976

Epoch 00003: val_loss did not improve from 0.13334
Epoch 4/10
376/376 [==============================] - 80s 212ms/step - loss: 0.3159 - accuracy: 0.8845 - val_loss: 0.3546 - val_accuracy: 0.8317

Epoch 00004: val_loss did not improve from 0.13334
Epoch 5/10
376/376 [==============================] - 79s 211ms/step - loss: 0.3138 - accuracy: 0.8950 - val_loss: 2.1579 - val_accuracy: 0.9366

Epoch 00005: val_loss did n